This notebook is simply to redeploy one of the models created from the `train_and_deploy-solution.ipynb` notebook. 

In [1]:
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor

Get the SageMaker execution role that this notebook is using and create the Predictor class.

In [2]:
role = sagemaker.get_execution_role()

In [3]:
jpeg_serializer = sagemaker.serializers.IdentitySerializer("image/jpeg")
json_deserializer = sagemaker.deserializers.JSONDeserializer()


class ImagePredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(ImagePredictor, self).__init__(
            endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=jpeg_serializer,
            deserializer=json_deserializer,
        )

We need to specify the path of the model output from one of the training jobs. You can do this by finding the file in your S3 bucket or checking the training jobs in the SageMaker console. There is probably a better way to do this using the SDK.

In [8]:
model_location = 's3://sagemaker-us-east-1-377574710449/dog-pytorch-2022-04-13-15-56-38-480/output/model.tar.gz'

In [9]:
pytorch_model = PyTorchModel(model_data=model_location, role=role, entry_point='infernce2.py',py_version='py3',
                             framework_version='1.4',
                             predictor_cls=ImagePredictor)

Now deploy the endpoint:

In [10]:
predictor = pytorch_model.deploy(initial_instance_count=1, instance_type='ml.m5.large')

------!

In [11]:
predictor.endpoint_name

'pytorch-inference-2022-04-15-14-32-59-360'

Once finished, make sure to delete the endpoint.

In [12]:
predictor.delete_endpoint()